In [5]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim # convert address into lotitude and longtitude values
from pandas.io.json import json_normalize # transform json file into pandas dataframe
import requests
# import matplitlib associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#
from sklearn.cluster import KMeans
import folium
print('libraries imported')

libraries imported


In [9]:
import urllib, json
url = 'https://cocl.us/new_york_dataset'
response = urllib.request.urlopen(url)
data = json.loads(response.read())
# print(data)

In [12]:
neighborhood_data = data['features']
# print(neighborhood_data)
neighborhood_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [57]:
# transform the data into pandas dataframe
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
neighborhoods = pd.DataFrame(columns = column_names)
for data in neighborhood_data:
    borough = neighborhood = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    neighborhood_latlong = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlong[1]
    neighborhood_long = neighborhood_latlong[0]
    neighborhoods = neighborhoods.append({'Borough': borough, 'Neighborhood': neighborhood_name, 
                                         'Latitude': neighborhood_lat, 'Longitude': neighborhood_long},
                                         ignore_index = True)
display(neighborhoods)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [21]:
# neighborhoods.head()
print(neighborhoods.describe(include = 'all'))

       Borough Neighborhood    Latitude   Longitude
count      306          306  306.000000  306.000000
unique       5          302         NaN         NaN
top     Queens    Sunnyside         NaN         NaN
freq        81            2         NaN         NaN
mean       NaN          NaN   40.701362  -73.946254
std        NaN          NaN    0.097498    0.121202
min        NaN          NaN   40.505334  -74.246569
25%        NaN          NaN   40.617778  -74.000062
50%        NaN          NaN   40.702969  -73.932009
75%        NaN          NaN   40.766645  -73.857525
max        NaN          NaN   40.908543  -73.708847


In [22]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


In [58]:
map_ny = folium.Map(location = [latitude, longitude], zoom_start = 10)
for lat, long, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], 
                                           neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, long], radius = 5,
                       popup = label, color = 'blue',
                       fill = True, fill_color = '#3186cc', fill_opacity = 0.7,
                       parse_html = False).add_to(map_ny)
map_ny

In [60]:
manhattan_data = neighborhoods.loc[(neighborhoods['Borough'] == 'Manhattan')].reset_index(drop = True)
manhattan_data.head(10)
# print(neighborhoods)
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)
manhattan_data

40.7896239 -73.9598939


,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


In [32]:
map_manh = folium.Map(location = [latitude, longitude], zoom_start = 11)
#
for lat, long, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, long], radius = 5, 
                       popup = label, color = 'blue', 
                       fill = True, fill_color = '#3186cc', foll_opacity = 0.7,
                       parse_html = False).add_to(map_manh)
map_manh

In [34]:
# use foursquare
CLIENT_ID = 'FU54GNAQI55REZXFB04YPT14WHILCMUSEJHRDEJCZ5V3G4OY'
CLIENT_SECRET = 'V1YTDUDWUYELBBFR25TGMJ4K52TKLLEC34XM0CTV3QK3GDHY'
VERSION = '20180605'
#
neighborhood_name = manhattan_data.loc[0, 'Neighborhood']
neighborhood_lat = manhattan_data.loc[0, 'Latitude']
neighborhood_long = manhattan_data.loc[0, 'Longitude']
print(neighborhood_name, neighborhood_lat, neighborhood_long)

Marble Hill 40.87655077879964 -73.91065965862981


In [36]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.\
format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_lat, neighborhood_long, radius, LIMIT)
#
results = requests.get(url).json()
# results

{'meta': {'code': 200, 'requestId': '5f0f0692ce241a5e036cd50e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [39]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3194, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym


In [40]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,69,69,69,69,69,69
Carnegie Hill,94,94,94,94,94,94
Central Harlem,48,48,48,48,48,48
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,44,44,44,44,44,44
East Village,100,100,100,100,100,100


In [41]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 325 uniques categories.


In [43]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()
# manhattan_onehot.shape

(3194, 326)

In [45]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped
# manhattan_grouped.shape

(40, 326)

In [46]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.10
1          Hotel  0.07
2            Gym  0.06
3    Coffee Shop  0.06
4  Memorial Site  0.04


----Carnegie Hill----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2  Pizza Place  0.04
3          Bar  0.04
4  Yoga Studio  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.06
1   Chinese Restaurant  0.06
2    French Restaurant  0.04
3  Fried Chicken Joint  0.04
4  American Restaurant  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.09
1          Art Gallery  0.06
2  American Restaurant  0.04
3    French Restaurant  0.03
4   Seafood Restaurant  0.03


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.09
1        Cocktail Bar  0.05
2              Bakery  0.05
3        Dessert Shop  0.03
4   Hotpot Restaurant  0.03


----Civic Center----
                  venue  freq
0           Coffee Shop  0.06
1  Gym / Fitness Center 

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Plaza,Gourmet Shop,Shopping Mall,Sandwich Place,BBQ Joint
1,Carnegie Hill,Coffee Shop,Café,Pizza Place,Bar,Bakery,Gym / Fitness Center,Gym,Italian Restaurant,Bookstore,Yoga Studio
2,Central Harlem,Chinese Restaurant,African Restaurant,Seafood Restaurant,American Restaurant,Bar,French Restaurant,Cosmetics Shop,Fried Chicken Joint,Dessert Shop,Cocktail Bar
3,Chelsea,Coffee Shop,Art Gallery,American Restaurant,French Restaurant,Bakery,Seafood Restaurant,Wine Shop,Cycle Studio,Café,Pizza Place
4,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,Salon / Barbershop,Vietnamese Restaurant,Spa,Bubble Tea Shop,Ice Cream Shop,Hotpot Restaurant,Optical Shop


In [59]:
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer

###### Cluster the Neighborhoods

In [49]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 2, 1, 2, 0, 1, 3, 2, 1])

In [50]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Gym,Sandwich Place,Coffee Shop,Yoga Studio,Pharmacy,Steakhouse,Shopping Mall,Seafood Restaurant,Pizza Place,Deli / Bodega
1,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Cocktail Bar,Bakery,Salon / Barbershop,Vietnamese Restaurant,Spa,Bubble Tea Shop,Ice Cream Shop,Hotpot Restaurant,Optical Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Bakery,Mobile Phone Shop,Deli / Bodega,Chinese Restaurant,Mexican Restaurant,Donut Shop,Spanish Restaurant,Italian Restaurant,Latin American Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,3,Mexican Restaurant,Lounge,Café,Restaurant,Park,Frozen Yogurt Shop,Bakery,Spanish Restaurant,Caribbean Restaurant,Chinese Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Sandwich Place,Sushi Restaurant,Bakery,Cocktail Bar,Yoga Studio


In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln Square,Plaza,Café,Concert Hall,Performing Arts Venue,Theater,American Restaurant,Italian Restaurant,Gym / Fitness Center,Gym,Wine Shop
21,Tribeca,Park,Italian Restaurant,American Restaurant,Wine Bar,Spa,Coffee Shop,Café,Men's Store,Poke Place,Bakery
28,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Plaza,Gourmet Shop,Shopping Mall,Sandwich Place,BBQ Joint
32,Civic Center,Coffee Shop,Hotel,Gym / Fitness Center,Spa,French Restaurant,Cocktail Bar,Italian Restaurant,Bakery,Yoga Studio,Park
35,Turtle Bay,Italian Restaurant,Coffee Shop,Sushi Restaurant,Wine Bar,Park,Seafood Restaurant,Deli / Bodega,French Restaurant,Hotel,Café
39,Hudson Yards,Hotel,American Restaurant,Gym / Fitness Center,Italian Restaurant,Café,Burger Joint,Dog Run,Coffee Shop,Gym,Park


In [53]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Gym,Sandwich Place,Coffee Shop,Yoga Studio,Pharmacy,Steakhouse,Shopping Mall,Seafood Restaurant,Pizza Place,Deli / Bodega
4,Hamilton Heights,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Sandwich Place,Sushi Restaurant,Bakery,Cocktail Bar,Yoga Studio
5,Manhattanville,Coffee Shop,Italian Restaurant,Deli / Bodega,Mexican Restaurant,Sushi Restaurant,Seafood Restaurant,Climbing Gym,Bar,Fried Chicken Joint,Scenic Lookout
8,Upper East Side,Italian Restaurant,Coffee Shop,Bakery,Gym / Fitness Center,Yoga Studio,Juice Bar,French Restaurant,Exhibit,Spa,Pizza Place
9,Yorkville,Italian Restaurant,Bar,Gym,Coffee Shop,Deli / Bodega,Pizza Place,Mexican Restaurant,Wine Shop,Sushi Restaurant,Japanese Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Cocktail Bar,Café,Deli / Bodega,Burger Joint,Gym,Gym / Fitness Center
12,Upper West Side,Italian Restaurant,Coffee Shop,Bar,Wine Bar,Dessert Shop,Bakery,Ice Cream Shop,Bookstore,Pizza Place,Middle Eastern Restaurant
14,Clinton,Italian Restaurant,Theater,Gym / Fitness Center,Wine Shop,American Restaurant,Coffee Shop,Sandwich Place,Gym,Cocktail Bar,Hotel
15,Midtown,Coffee Shop,Hotel,Clothing Store,Theater,Bakery,Steakhouse,Pizza Place,Bookstore,Japanese Restaurant,Gym
16,Murray Hill,Sandwich Place,Japanese Restaurant,Coffee Shop,Burger Joint,Italian Restaurant,American Restaurant,Bar,Pizza Place,Hotel,Gym / Fitness Center


In [54]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,Salon / Barbershop,Vietnamese Restaurant,Spa,Bubble Tea Shop,Ice Cream Shop,Hotpot Restaurant,Optical Shop
6,Central Harlem,Chinese Restaurant,African Restaurant,Seafood Restaurant,American Restaurant,Bar,French Restaurant,Cosmetics Shop,Fried Chicken Joint,Dessert Shop,Cocktail Bar
19,East Village,Bar,Ice Cream Shop,Pizza Place,Mexican Restaurant,Cocktail Bar,Coffee Shop,Korean Restaurant,Wine Bar,Italian Restaurant,Dessert Shop
20,Lower East Side,Chinese Restaurant,Cocktail Bar,Japanese Restaurant,Bakery,Pizza Place,Ramen Restaurant,Art Gallery,Coffee Shop,Mediterranean Restaurant,French Restaurant
25,Manhattan Valley,Bar,Coffee Shop,Yoga Studio,Mexican Restaurant,Thai Restaurant,Pizza Place,Bubble Tea Shop,Café,Caribbean Restaurant,Chinese Restaurant
27,Gramercy,Bar,American Restaurant,Coffee Shop,Bagel Shop,Italian Restaurant,Diner,Cocktail Bar,Ice Cream Shop,Mexican Restaurant,Thai Restaurant
33,Midtown South,Korean Restaurant,Hotel,Japanese Restaurant,Dessert Shop,Bakery,Burger Joint,Gym / Fitness Center,Clothing Store,Coffee Shop,Pizza Place


In [55]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Deli / Bodega,Chinese Restaurant,Mexican Restaurant,Donut Shop,Spanish Restaurant,Italian Restaurant,Latin American Restaurant
3,Inwood,Mexican Restaurant,Lounge,Café,Restaurant,Park,Frozen Yogurt Shop,Bakery,Spanish Restaurant,Caribbean Restaurant,Chinese Restaurant
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Pharmacy,Latin American Restaurant,Deli / Bodega,Park,Sandwich Place,Spa,Gas Station
11,Roosevelt Island,Park,Deli / Bodega,Soccer Field,School,Scenic Lookout,Noodle House,Sandwich Place,Liquor Store,Dry Cleaner,Baseball Field
26,Morningside Heights,Park,Bookstore,American Restaurant,Coffee Shop,Deli / Bodega,Burger Joint,Supermarket,Outdoor Sculpture,Optical Shop,Salad Place
36,Tudor City,Café,Park,Mexican Restaurant,Deli / Bodega,Sushi Restaurant,Greek Restaurant,Pizza Place,Spanish Restaurant,Dog Run,Coffee Shop


In [56]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Park,Bar,Pet Service,Baseball Field,Gym / Fitness Center,Cocktail Bar,Harbor / Marina,Coffee Shop,Skating Rink,Heliport
